In [ ]:
!pip install geopandas
!pip install turfpy
!pip install contextily

from google.colab import files
uploaded = files.upload()

     |████████████████████████████████| 1.0 MB 4.3 MB/s 
     |████████████████████████████████| 16.7 MB 347 kB/s 
     |████████████████████████████████| 6.3 MB 33.7 MB/s 
  Created wheel for turfpy: filename=turfpy-0.0.7-py3-none-any.whl size=39119 sha256=c471e8e27d93d377e9378f0b061bdd6b6b485791d6c7e2c2810dda1ee944297f
  Stored in directory: /root/.cache/pip/wheels/56/d0/d8/13858feab5280185516cd22df1adc0f75426968815b057dc5e
Successfully built turfpy
     |████████████████████████████████| 19.3 MB 68.2 MB/s 


Saving 20_noon.CSV to 20_noon.CSV
Saving 20_eve.CSV to 20_eve.CSV
Saving 14_morning.CSV to 14_morning.CSV
Saving grid2.geojson to grid2.geojson


In [ ]:

import json
import glob
import numpy as np
import math
import folium
from folium import plugins

import io
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (10, 15)
file_name="grid2.geojson"

df_places = gpd.read_file(file_name)
# ax = df_places.plot(color='blue')
df_places

,left,bottom,right,top,geometry
0,77.845,29.866343,77.851667,29.872117,"MULTIPOLYGON (((77.84500 29.86923, 77.84667 29..."
1,77.845,29.872117,77.851667,29.877890,"MULTIPOLYGON (((77.84500 29.87500, 77.84667 29..."
2,77.845,29.877890,77.851667,29.883664,"MULTIPOLYGON (((77.84500 29.88078, 77.84667 29..."
3,77.850,29.869230,77.856667,29.875004,"MULTIPOLYGON (((77.85000 29.87212, 77.85167 29..."
4,77.850,29.875004,77.856667,29.880777,"MULTIPOLYGON (((77.85000 29.87789, 77.85167 29..."
...,...,...,...,...,...
110,77.910,29.880777,77.916667,29.886551,"MULTIPOLYGON (((77.91000 29.88366, 77.91167 29..."
111,77.910,29.886551,77.916667,29.892324,"MULTIPOLYGON (((77.91000 29.88944, 77.91167 29..."
112,77.915,29.866343,77.921667,29.872117,"MULTIPOLYGON (((77.91500 29.86923, 77.91667 29..."
113,77.915,29.877890,77.921667,29.883664,"MULTIPOLYGON (((77.91500 29.88078, 77.91667 29..."


In [ ]:
df_places['hex_id']=range(0,len(df_places))
geojson=json.loads(uploaded[file_name].decode("utf-8"))
from pandas.io.json import json_normalize
df = json_normalize(geojson["features"])

coords = 'geometry.coordinates'

df2 = (df[coords].apply(lambda r: [(i[0],i[1],i[2],i[3],i[4],i[5]) for i in r[0]])
           .apply(pd.Series).stack()
           .reset_index(level=1).rename(columns={0:coords,"level_1":"point"})
           .join(df.drop(coords,1), how='left')).reset_index(level=0)

df2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # This is added back by InteractiveShellApp.init_path()


,index,point,geometry.coordinates,type,properties.left,properties.bottom,properties.right,properties.top,geometry.type
0,0,0,"([77.845, 29.86923010305707], [77.846666667443...",Feature,77.845,29.866343,77.851667,29.872117,MultiPolygon
1,1,0,"([77.845, 29.875003608440863], [77.84666666744...",Feature,77.845,29.872117,77.851667,29.877890,MultiPolygon
2,2,0,"([77.845, 29.880777113824657], [77.84666666744...",Feature,77.845,29.877890,77.851667,29.883664,MultiPolygon
3,3,0,"([77.85000000233126, 29.872116855748967], [77....",Feature,77.850,29.869230,77.856667,29.875004,MultiPolygon
4,4,0,"([77.85000000233126, 29.87789036113276], [77.8...",Feature,77.850,29.875004,77.856667,29.880777,MultiPolygon
...,...,...,...,...,...,...,...,...,...
110,110,0,"([77.91000003030626, 29.883663866516557], [77....",Feature,77.910,29.880777,77.916667,29.886551,MultiPolygon
111,111,0,"([77.91000003030626, 29.88943737190035], [77.9...",Feature,77.910,29.886551,77.916667,29.892324,MultiPolygon
112,112,0,"([77.91500003263752, 29.86923010305707], [77.9...",Feature,77.915,29.866343,77.921667,29.872117,MultiPolygon
113,113,0,"([77.91500003263752, 29.880777113824657], [77....",Feature,77.915,29.877890,77.921667,29.883664,MultiPolygon


In [ ]:
# csv_files=glob.glob('/content/drive/MyDrive/AQIData/2022-03-08/*.json')
# from datetime import datetime
# from itertools import repeat

# for i,file in enumerate(csv_files):
#   date_for_file = file.split('_')[-1].split('.')[0]
#   date_for_file=date_for_file[:11]+' '+date_for_file[12:]
#   date_for_file=date_for_file[:14]+':'+date_for_file[15:]
#   date_for_file= pd.to_datetime(date_for_file)
#   df = pd.read_csv(file)
#   ind=df.shape[0]
#   df['Time'] = date_for_file
#   df.Time = pd.to_datetime(df.Time, format='%Y-%m-%d %H:%M:%S')
#   df['minute'] = df.Time.apply(lambda x: x.hour)*60+df.Time.apply(lambda x: x.minute)
#   if i == 0:
#    data = pd.DataFrame()
#    data = df
#   else:
#    data = data.append(df)
# data.reset_index(inplace = True)
# csv_files=glob.glob('/content/*.CSV')
# for i,file in enumerate(csv_files):
#   df = pd.read_csv(file)
#   if i == 0:
#    data = pd.DataFrame()
#    data = df
#   else:
#    data = data.append(df)
# data.reset_index(inplace = True)
# data["hex_id"]=0

data1=pd.read_csv("14_morning.CSV")
data2=pd.read_csv("20_noon.CSV")
data3=pd.read_csv("20_eve.CSV")

In [ ]:
#finding centroids
centreLat=[]
centreLon=[]
for i in df2.index:
  centreLat.append(float(df2['geometry.coordinates'][i][0][1]+df2['geometry.coordinates'][i][3][1])/2)
  centreLon.append(float(df2['geometry.coordinates'][i][0][0]+df2['geometry.coordinates'][i][3][0])/2)
df_places['LatCen']=centreLat
df_places['LonCen']=centreLon

In [ ]:
from turfpy.measurement import boolean_point_in_polygon
from geojson import Point, Polygon, Feature
def hexagonal(data):
  for ind in data.index:
      x=data.loc[ind,"latitude"]
      y=data.loc[ind,"longitude"]
      # z=data.loc[ind,"isOnline"]
      if(math.isnan(x) or math.isnan(y)):
          continue
      
      for j in df2.index:
        point = Feature(geometry=Point((x,y)))
        polygon = Polygon(
            [
            [
              (df2['geometry.coordinates'][j][0][1],df2['geometry.coordinates'][j][0][0]),
              (df2['geometry.coordinates'][j][1][1],df2['geometry.coordinates'][j][1][0]),
              (df2['geometry.coordinates'][j][2][1],df2['geometry.coordinates'][j][2][0]),
              (df2['geometry.coordinates'][j][3][1],df2['geometry.coordinates'][j][3][0]),
              (df2['geometry.coordinates'][j][4][1],df2['geometry.coordinates'][j][4][0]),
              (df2['geometry.coordinates'][j][5][1],df2['geometry.coordinates'][j][5][0]),
              ]
            ])
        if(boolean_point_in_polygon(point,polygon)):
          data.loc[ind,"hex_id"]=j
          # r=result_df.loc[ind,"PM25"]
          # pm[j]+=r
          # count[j]+=1

  result_df=data[data["hex_id"]!=0]
  return result_df

In [ ]:
def interpolation(result_df):
  daf = result_df.copy()
  daf['Time'] = pd.to_datetime(daf['Data'] + ' ' + daf['Time'])
  df_interpol = daf.groupby('hex_id')\
                  .resample('T',on='Time')\
                  .mean()
  df_interpol['PM2.5'] = df_interpol['PM2.5'].interpolate()
  del df_interpol['hex_id']
  daf1=df_interpol.reset_index(level=['Time'])
  # df_interpol.Time = pd.to_datetime(df_interpol.Time, format='%Y-%m-%d %H:%M:%S')
  daf1['minute'] = daf1.Time.apply(lambda x: x.hour)*60+daf1.Time.apply(lambda x: x.minute)
  return daf1

In [ ]:
daf1=interpolation(hexagonal(data1))
daf2=interpolation(hexagonal(data2))
daf3=interpolation(hexagonal(data3))

In [ ]:
daf1=daf1.append(daf2)

In [ ]:
daf1=daf1.append(daf3)
daf1

,Time,latitude,longitude,PM1,PM2.5,PM10,CO2,SO2,NO2,CO,O3,TEMP.,HUMI.,minute,H2S
hex_id,,,,,,,,,,,,,,,
42.0,2022-08-03 10:06:00,29.871588,77.876360,114.000000,140.000000,159.000000,0.0,0.000,0.000,0.000,0.000000,25.300000,53.800000,606,NaN
42.0,2022-08-03 10:07:00,29.872109,77.876538,111.000000,136.000000,156.000000,0.0,0.000,0.000,0.000,0.000000,25.400000,53.300000,607,NaN
43.0,2022-08-03 10:09:00,29.877903,77.876548,133.000000,164.000000,188.000000,0.0,0.000,0.000,0.000,0.000000,26.000000,50.400000,609,NaN
53.0,2022-08-03 10:03:00,29.864584,77.878564,117.600000,144.600000,165.400000,0.0,0.000,0.000,0.000,0.000000,24.500000,55.820000,603,NaN
53.0,2022-08-03 10:04:00,29.865983,77.878639,129.500000,159.250000,182.500000,0.0,0.000,0.000,0.000,0.000000,24.950000,54.925000,604,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113.0,2022-08-03 18:30:00,29.881144,77.917498,68.000000,82.400000,85.400000,0.0,0.025,-0.023,0.250,-0.023000,25.740000,53.120000,1110,0.0
113.0,2022-08-03 18:31:00,29.881256,77.919252,82.600000,100.000000,104.800000,0.0,0.025,-0.023,0.244,-0.022200,25.680000,53.340000,1111,0.0
113.0,2022-08-03 18:32:00,NaN,NaN,NaN,109.264706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1112,NaN


In [ ]:
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

In [ ]:
import contextily as cx
def plot_choropleth(timestep):
    # Get subset of dataframe

    df3=daf1[daf1.minute==timestep]
    df4=df3.groupby('hex_id')
    df4=df4.aggregate(np.mean)
    merged = df_places.set_index('hex_id').join(df4)
  
    for ind in merged.index:
      if(math.isnan(merged.loc[ind,"PM2.5"])):
        merged.loc[ind,"PM2.5"]=0

    myLon=[]
    myLat=[]
    myPM=[]
    count=0
    for ind in merged.index:
      if(merged.loc[ind,"PM2.5"]!=0):
        myLat.append(merged.loc[ind,"LatCen"])
        myLon.append(merged.loc[ind,"LonCen"])
        myPM.append(merged.loc[ind,"PM2.5"])
        count+=1
    if(count==0):
      return 0
    
    for ind in merged.index:
      if(merged.loc[ind,"PM2.5"]!=0 or count==0):
        continue
      currLat=merged.loc[ind,"LatCen"]
      currLon=merged.loc[ind,"LonCen"]
      sum=0
      mul=0
      n=len(myLon)
      for i in range(0,n):
        
        distance=(myLon[i]-currLon)*(myLon[i]-currLon)+(myLat[i]-currLat)*(myLat[i]-currLat)
        weight=1/distance
        sum+=weight
        mul+=(weight)*myPM[i]
      
      if(n==1):
        r=((myLon[0]-currLon)**2+(myLat[0]-currLat)**2)**0.5
        merged.loc[ind,"PM2.5"]=myPM[0]/10000**r
      else:
        merged.loc[ind,"PM2.5"]=round(mul/sum)
    merged = merged.to_crs(epsg=3857)
    # set the range for the choropleth
    vmin, vmax =0,300
    # create figure and axes for Matplotlib
    norm=plt.Normalize(vmin=vmin,vmax=vmax)
    fig, ax = plt.subplots(1, figsize=(15, 10))
    # create map
    colorlist = ["green", "yellow", "orange", "red","purple"]
    newcmp = LinearSegmentedColormap.from_list('testCmap', colors=colorlist, N=256)
    ax2 = merged.plot(column='PM2.5', 
                      cmap=newcmp, 
                      linewidth=0.1, 
                      ax=ax, 
                      edgecolor='black',
                      norm=norm, 
                      alpha=0.8)
    # add basemap
    cx.add_basemap(ax2, zoom=11,          
                    source=cx.providers.OpenStreetMap.Mapnik)
    # Remove axis labels
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])
    # add a title
    hour=str(int(timestep/60))
    min=str(int((timestep%60)/10))+str((timestep%60)%10)
    time=hour+":"+min
    ax.set_title('PM 2.5\nTime:- {}'.format(time),
                 fontdict={'fontsize': '20', 'fontweight' : '3'})
    # Create colorbar as a legend
    sm = plt.cm.ScalarMappable(cmap=newcmp,    
                norm=plt.Normalize(vmin=0, vmax=300))
    sm._A = []
    cbar = fig.colorbar(sm)
    return ax, fig

In [ ]:
!rm -rf viz

In [ ]:
!mkdir viz
total_timesteps = int(daf1.describe()['minute']['max'])
j=0
for i in range(total_timesteps):
    if(plot_choropleth(i)==0):
      continue
    j=j+1
    ax, fig = plot_choropleth(i)

    fig.savefig('./viz/img{:03d}.png'.format(j), 
              dpi=100, format='png',
              bbox_inches='tight')
    plt.close()

In [ ]:
%%bash
cd viz;
# Create a gif viz
ffmpeg -framerate 2 -i img%03d.png -f gif out.gif

In [ ]:
%%bash
cd viz;

# Create mp4 video
ffmpeg -framerate 2 -i img%03d.png -c:v libx264 -pix_fmt yuv420p -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" out.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib